In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms, models
# import torchvision.models as models
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  
])

In [4]:

model = models.convnext_tiny(weights= None)

In [ ]:
# print(model)

In [5]:
model.classifier[2] = nn.Linear(768,50)

In [6]:
import torchinfo

torchinfo.summary(model, (32, 3,64,64))

Layer (type:depth-idx)                        Output Shape              Param #
ConvNeXt                                      [32, 50]                  --
├─Sequential: 1-1                             [32, 768, 2, 2]           --
│    └─Conv2dNormActivation: 2-1              [32, 96, 16, 16]          --
│    │    └─Conv2d: 3-1                       [32, 96, 16, 16]          4,704
│    │    └─LayerNorm2d: 3-2                  [32, 96, 16, 16]          192
│    └─Sequential: 2-2                        [32, 96, 16, 16]          --
│    │    └─CNBlock: 3-3                      [32, 96, 16, 16]          79,296
│    │    └─CNBlock: 3-4                      [32, 96, 16, 16]          79,296
│    │    └─CNBlock: 3-5                      [32, 96, 16, 16]          79,296
│    └─Sequential: 2-3                        [32, 192, 8, 8]           --
│    │    └─LayerNorm2d: 3-6                  [32, 96, 16, 16]          192
│    │    └─Conv2d: 3-7                       [32, 192, 8, 8]           73,920

In [ ]:
# model = CNN(num_classes=50).to(device)

In [7]:
train_dir = '/kaggle/input/iith-dl-contest-2024/train/train'
train_data  = datasets.ImageFolder(train_dir, transform = transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

In [8]:
model.load_state_dict(torch.load("/kaggle/input/convexnexttiny/weights10.pth"))

# # Defining loss criterion and optimizer
# loss_func = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Training loop
# epochs = 16
# for epoch in range(epochs):
#     model.train()  # Setting the model to training mode
#     running_loss = 0.0
#     correct_preds = 0
#     total_samples = 0
#     for i, (inputs, labels) in tqdm(enumerate(train_loader)):
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()  # Clearing gradients

#         # Forward pass
#         outputs = model(inputs)

#         # Computing loss
#         loss = loss_func(outputs, labels)

#         # Backward pass and optimization
#         loss.backward()
#         optimizer.step()

#         # Updating running loss
#         running_loss += loss.item() * inputs.size(0)

#         # Calculating accuracy
#         _, predicted = torch.max(outputs, 1)
#         correct_preds += torch.sum(predicted == labels).item()
#         total_samples += labels.size(0)

#     # Printing average loss and accuracy for the current epoch
#     epoch_loss = running_loss / len(train_data)
#     epoch_acc = correct_preds / total_samples
#     print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')
#     if epoch%2 + 1==2:
#         str = "/kaggle/working/weights{}.pth".format(epoch + 1)
#         torch.save(model.state_dict(), str)
        
    

<All keys matched successfully>

In [9]:
test_dir = '/kaggle/input/iith-dl-contest-2024/test'
test_data  = datasets.ImageFolder(test_dir, transform = transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

In [10]:
import numpy as np
import csv

In [11]:
# Reverse the class to index mapping to index to class for prediction interpretation
classes = train_data.class_to_idx
idx_to_class = {idx: class_name for class_name, idx in classes.items()}

In [12]:
# List to store outputs
outputs_list = []

# Process images and gather predictions
for images, _ in tqdm(test_loader):
    images = images.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs, dim=1)
    outputs_list.append(predicted)

# Concatenate all predictions into a single tensor
outputs = torch.cat(outputs_list).cpu().numpy()

100%|██████████| 1199/1199 [04:13<00:00,  4.73it/s]


In [13]:
import numpy as np

# Convert class indices to class names
predicted_classes = np.array([idx_to_class[idx] for idx in outputs], dtype=object)

# Generate image file names
file_names = [f"{i}.JPEG" for i in range(len(predicted_classes))]

file_names = np.sort(file_names)

In [14]:
# Combine file names and predicted classes
table = np.column_stack((file_names, predicted_classes))

# Print table (optional, can be commented out in production)
print(table)

[['0.JPEG' 'n02808440']
 ['1.JPEG' 'n02099712']
 ['10.JPEG' 'n01944390']
 ...
 ['9997.JPEG' 'n02480495']
 ['9998.JPEG' 'n02395406']
 ['9999.JPEG' 'n01784675']]


In [15]:
# Write results to CSV file
with open('submission1.csv', 'w', newline='') as file:
    wr = csv.writer(file)
    wr.writerow(['ID', 'Category'])
    wr.writerows(table)

In [ ]:
# torch.save(model.state_dict(),"/kaggle/working/model_weights_40.pth")